In [1]:
!pip install langchain_openai langchain langchain_community langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
import os
os.environ['OPENAI_API_KEY']="sk-proj-fJvv-gqb8r"
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = "lsv2_pt_b7be9"
os.environ['LANGCHAIN_PROJECT'] = "chat_bot_with_memory"

In [40]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory   # the entire conservation is managed inside the list
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder


In [41]:
llm_model = ChatOpenAI(model= 'gpt-3.5-turbo')
parser = StrOutputParser()

In [42]:
# simple qa
while True:
  message = input("Write your query:")
  if message=="bye":
    print("Bye have a great day!")
    break
  else:
    print(parser.invoke(llm_model.invoke([HumanMessage(content=message)])))

Write your query:hi
Hello! How can I assist you today?
Write your query:bye
Bye have a great day!


In [43]:
# making the list of the model so model has memory
output = llm_model.invoke(
    [
    HumanMessage(content = "hi i am ibtisam asghar"),
    AIMessage(content = 'Hello ibtisam asghar, how can i help you?'),
    HumanMessage(content = "What is my name can you tell me ?")
    ]
)

In [44]:
parser.invoke(output)

"Your name is ibtisam asghar. Is there anything else you'd like to know or talk about?"

In [45]:
store = {} # this dictionary will act like a storage

def get_session_history (session_id : str) -> BaseChatMessageHistory :
  if session_id not in store :
    store[session_id] = InMemoryChatMessageHistory() # Creates a new InMemoryChatMessagHistory() and store it under that session_id
  return store[session_id] # if already presnt just reuse this existing history

In [48]:

configuration = {'configurable':{"session_id" : "myfirstchat"}}

In [49]:
model_with_memory = RunnableWithMessageHistory(llm_model , get_session_history) # wrapper in langchain allows us to add the chat history tracking to any runnable (like an LLM Chain, an agent etc)

In [50]:
model_with_memory.invoke([HumanMessage(content = 'Hi i am ibtisam')] , config= configuration).content

'Hello Ibtisam, nice to meet you! How can I assist you today?'

In [51]:
model_with_memory.invoke([HumanMessage(content = 'What is my name?')] , config= configuration).content

'Your name is Ibtisam.'

In [52]:
configuration = {'configurable':{"session_id" : "mysecondchat"}}

In [53]:
configuration

{'configurable': {'session_id': 'mysecondchat'}}

In [54]:
model_with_memory.invoke([HumanMessage(content = 'What is my name?')] , config= configuration).content

"I'm sorry, I don't have access to personal information about users. You can simply call me Assistant. How can I assist you today?"

# Creating the ChatPromptTemplate

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system','You are a helpful assistant carefully analyze the user query and assist the user with the answer of user question tp the best of your ability.',),
        MessagesPlaceholder(variable_name="messages") # nsert all the past conservation history here.
    ]
)

In [17]:
chain = prompt | llm_model | parser

In [18]:
chain.invoke({'messages' : [HumanMessage(content = 'Hi i am ibtisam')]})

'Hello, Ibtisam! How can I assist you today?'

In [19]:
chain.invoke({'messages' : [HumanMessage(content = 'What is my name')]})

'I apologize, but as an AI assistant, I do not have access to your personal information like your name. If you would like to share your name with me, feel free to do so. How else can I assist you today?'

In [20]:
model_with_memory = RunnableWithMessageHistory(chain,get_session_history)

In [21]:
configuration = {'configurable':{"session_id" : "mythirdchat"}}

In [22]:
response = model_with_memory.invoke([HumanMessage(content = 'Hi i am ibtisam')],config=configuration)

In [23]:
response

'Hello Ibtisam! How can I assist you today?'

In [24]:
response = model_with_memory.invoke([HumanMessage(content = 'What is my name')],config=configuration)

In [25]:
response

'Your name is Ibtisam.'

In [26]:
store

{'myfirstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi i am ibtisam', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Ibtisam! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 14, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C7IjchQ8b2Cj3GOYUqTxLOvjMRMlf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ad9a241c-3f1f-4df9-89ce-557a43b2924a-0', usage_metadata={'input_tokens': 14, 'output_tokens': 13, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='What is my n

In [27]:
response=model_with_memory.invoke(
    [HumanMessage(content="what is 2+2?"),
     ],config=configuration
)
response

'2 + 2 equals 4.'

In [28]:
response=model_with_memory.invoke(
    [HumanMessage(content="what should i add in the preveious addition so that it will become 11"),
     ],config=configuration
)
response

'To turn the previous addition of 2 + 2 into 11, you would need to add 7. Adding 7 to 4 results in 11.'

# Trimming the conservation

In [29]:
from langchain_core.messages import SystemMessage , trim_messages

In [30]:
trimmer = trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter= llm_model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)
messages = [
    HumanMessage(content="hi! I'm Ibtisam"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like coding"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [31]:
llm_model.get_num_tokens_from_messages(messages)

74

In [32]:
trimmer.invoke(messages)

[HumanMessage(content='I like coding', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [33]:
trimmed_message = trimmer.invoke(messages)

In [34]:
llm_model.get_num_tokens_from_messages(trimmed_message)

56

In [35]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | llm_model | parser
)


In [36]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what is my name?")],
        "language": "English",
    }
)
response

"I'm sorry, I don't have access to personal information like your name. Is there anything else I can assist you with?"

In [37]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response

'You asked "what\'s 2 + 2?"'